In [1]:
from transformers import BertTokenizer, TFDistilBertForSequenceClassification
from transformers import TFDistilBertModel
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from transformers import BertTokenizer, TFDistilBertForSequenceClassification
import numpy as np
from dataset_vars_load import *
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf

/opt/anaconda3/envs/my_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-12 19:45:40.200050: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load the pre-trained model
model_name = "distilbert-base-uncased"
bert_model = TFDistilBertModel.from_pretrained(model_name)

/opt/anaconda3/envs/my_env/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-08-12 19:46:26.555286: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initia

In [3]:
# Define the classification head

tokenizer = BertTokenizer.from_pretrained(model_name)

num_labels = len(data['label'].unique())  # Assuming unique labels represent number of classes

model = TFDistilBertForSequenceClassification.from_pretrained(
    model_name, num_labels=num_labels
)

print(num_labels)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from 

7


In [4]:
# Fine-tuning
optimizer = Adam(learning_rate=0.001)
loss = CategoricalCrossentropy()
metrics = ['accuracy']
batch_size = 32
epochs = 1

In [5]:
# Feature extraction using TF-IDF with max 500 features
vectorizer = TfidfVectorizer(max_features=500, ngram_range=(1, 2))
X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.transform(X_test)

In [6]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
model.fit(X_train_features.toarray().astype(int), tf.keras.utils.to_categorical(y_train), batch_size=batch_size, epochs=epochs, verbose=1)

2024-08-12 19:48:09.471817: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (mklcpu) ran out of memory trying to allocate 91.55MiB (rounded to 96000000)requested by op tf_distil_bert_for_sequence_classification/distilbert/transformer/layer_._2/attention/dropout_27/dropout/GreaterEqual
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-08-12 19:48:09.658552: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for mklcpu
2024-08-12 19:48:09.671720: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-08-12 19:48:09.672976: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested